In [1]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol, get_vdw_radius, add_charges_to_molecule, get_node_features, get_edge_features, extract_charges_from_mol2
from src.utils.constants import ES_THRESHOLD


In [2]:
def ionic_interaction(mol: Chem.Mol, atom_1: int, atom_2: int):
    """
    Determine if there is an electrostatic interaction between atom 1 and atom 2 based on whether the coulombic interaction
    between the two atoms is less than or equal to some threshold imported from src/utils/constants.py.

    Parameters:
    mol (Chem.Mol): The RDKit molecule. NOTE: We assume that the molecule has _TriposAtomCharges charges assigned to the atoms.
    atom_1 (int): The index of the first atom.
    atom_2 (int): The index of the second atom.
    
    Returns:
    tuple: (0, 0, 0, 0, 0) if there is an electrostatic interaction, None otherwise 

    Conversion Factor Calculation:
    The Coulombic interaction energy between two point charges q1 and q2 separated by a distance r in vacuum is given by:

        E = k * q1 * q2 / r

    Where:
    - E is the energy in Joules (J).
    - k is Coulomb's constant, 8.987 x 10^9 N m^2 C^-2.
    - q1 and q2 are the charges in Coulombs (C).
    - r is the distance in meters (m).

    To convert this energy into kJ/mol, the following conversions are applied:
    - 1 e (elementary charge) = 1.602 x 10^-19 C
    - Avogadro's number (N_A) = 6.022 x 10^23 mol^-1
    - 1 Angstrom (Å) = 10^-10 meters (m)

    Therefore, the conversion factor from (charge_1 * charge_2) / distance to kJ/mol is:

        Conversion factor = (k * N_A * (1.602 x 10^-19)^2) * 10^10

    Plugging in the values:

        Conversion factor = (8.987 x 10^9 * 6.022 x 10^23 * (1.602 x 10^-19)^2) * 10^10
                          ≈ 1388.93 kJ mol^-1 Å^-1
    """
    # Constants
    CONVERSION_FACTOR_KJ = 1388.93  # kJ mol^-1 Å^-1

    # Get atomic charges using Gasteiger charges or another suitable method
    charge_1 = float(mol.GetAtomWithIdx(atom_1).GetProp('_TriposAtomCharges'))
    charge_2 = float(mol.GetAtomWithIdx(atom_2).GetProp('_TriposAtomCharges'))
    
    # Get the distance between the two atoms
    conf = mol.GetConformer()
    pos_1 = conf.GetAtomPosition(atom_1)
    pos_2 = conf.GetAtomPosition(atom_2)
    distance = pos_1.Distance(pos_2)

    coulombic_interaction_kj = (charge_1 * charge_2 / distance) * CONVERSION_FACTOR_KJ

    # Check if the Coulombic interaction is less than or equal to the threshold
    if abs(coulombic_interaction_kj) >= ES_THRESHOLD:
        return (0, 0, 0, 0, 0, coulombic_interaction_kj)
    else:
        return None

In [3]:
def calculate_gasteiger_charges(mol):
    AllChem.ComputeGasteigerCharges(mol)
    for atom in mol.GetAtoms():
        atom.SetProp('_TriposAtomCharges', str(atom.GetDoubleProp('_GasteigerCharge')))
    return mol


In [4]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  None
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    mol = calculate_gasteiger_charges(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    result = ionic_interaction(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Result: {result}")


SMILES: C=CCCCCCCCCC, Atoms: (0, 10) -> Result: None
SMILES: C=C, Atoms: (0, 1) -> Result: None
SMILES: CC(=O)O, Atoms: (2, 3) -> Result: (0, 0, 0, 0, 0, 74.26637744171447)


[21:40:57] Molecule does not have explicit Hs. Consider calling AddHs()
[21:40:58] Molecule does not have explicit Hs. Consider calling AddHs()
[21:40:58] Molecule does not have explicit Hs. Consider calling AddHs()


In [13]:
complex_mol = mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2", sanitize = False) #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
ligand_mol = mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_ligand.mol2", sanitize = False) #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
num_atoms_in_protein = len(complex_mol.GetAtoms()) - len(ligand_mol.GetAtoms())
charges = extract_charges_from_mol2("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2")

mol = complex_mol
AssignStereochemistry(mol, force=True, cleanIt=True)
protein_or_ligand_ids = [-1 if atom.GetIdx() < num_atoms_in_protein else 1 for atom in mol.GetAtoms()]

In [6]:
add_charges_to_molecule(mol, charges)
#pocket_atom_indices = find_pocket_atoms_RDKit(mol, protein_or_ligand_ids, num_atoms_in_protein) # just use all of the atoms for now 
pocket_atom_indices = [atom.GetIdx() for atom in mol.GetAtoms()]
edge_features, edge_indices = get_edge_features(mol, pocket_atom_indices, pairwise_function=ionic_interaction)

6349it [01:31, 69.40it/s] 


In [10]:
# Extract the last column from edge_features
last_column = edge_features[:, -1]

# Sort edge_indices based on the last column
sorted_indices = np.argsort(last_column)
sorted_edge_indices = edge_indices[sorted_indices]
sorted_edge_features = edge_features[sorted_indices]

print("Sorted Edge Indices:", sorted_edge_indices[:100])

Sorted Edge Indices: [[1241 1243]
 [1243 1241]
 [2807 2805]
 [2805 2807]
 [2806 2805]
 [2805 2806]
 [4606 4608]
 [4608 4606]
 [1241 1242]
 [1242 1241]
 [2301 2302]
 [2302 2301]
 [1059 1060]
 [1060 1059]
 [2301 2303]
 [2303 2301]
 [4359 4357]
 [4357 4359]
 [1597 1596]
 [1596 1597]
 [1598 1596]
 [1596 1598]
 [2580 2581]
 [2581 2580]
 [4607 4606]
 [4606 4607]
 [1683 1685]
 [1685 1683]
 [1061 1059]
 [1059 1061]
 [3326 3328]
 [3328 3326]
 [2968 2967]
 [2967 2968]
 [4094 4095]
 [4095 4094]
 [4094 4096]
 [4096 4094]
 [3941 3939]
 [3939 3941]
 [2969 2967]
 [2967 2969]
 [4788 4786]
 [4786 4788]
 [2536 2535]
 [2535 2536]
 [3939 3940]
 [3940 3939]
 [3682 3684]
 [3684 3682]
 [2506 2508]
 [2508 2506]
 [4786 4787]
 [4787 4786]
 [ 381  382]
 [ 382  381]
 [6227 6228]
 [6228 6227]
 [2507 2506]
 [2506 2507]
 [3350 3349]
 [3349 3350]
 [2843 2841]
 [2841 2843]
 [1683 1684]
 [1684 1683]
 [5384 5385]
 [5385 5384]
 [4358 4357]
 [4357 4358]
 [1864 1862]
 [1862 1864]
 [1863 1862]
 [1862 1863]
 [5384 5386]
 [53

In [9]:
print("Sorted Edge Features:", sorted_edge_features)

Sorted Edge Features: [[   0.            0.            0.            0.            0.
  -133.37202442]
 [   0.            0.            0.            0.            0.
  -133.37202442]
 [   0.            0.            0.            0.            0.
  -127.02707769]
 ...
 [   0.            0.            0.            0.            0.
    56.4821499 ]
 [   0.            0.            0.            0.            0.
    57.54894827]
 [   0.            0.            0.            0.            0.
    57.54894827]]
